In [ ]:
from rembg import remove
from PIL import Image
import numpy as np
import random
import cv2

def replace_background(input_path, bg_path, output_path):

    image = cv2.imread(input_path)
    height, width = image.shape[:2]
    # Xác định độ dịch chuyển ngẫu nhiên
    shift_x = random.randint(-50, 50)  # Dịch sang trái hoặc phải ngẫu nhiên
    # Ma trận dịch chuyển (translation matrix)
    M = np.float32([[1, 0, shift_x], [0, 1, 0]])
    # Áp dụng transformation
    shifted_image = cv2.warpAffine(image, M, (width, height))
    cv2.imwrite(input_path,shifted_image)
    
    # Đọc ảnh gốc
    image = Image.open(input_path).convert("RGBA")
    
    # Xóa nền cũ
    removed_bg = remove(image)

    # Đọc ảnh nền mới
    bg_image = Image.open(bg_path).convert("RGBA").resize(image.size)

    # Chuyển ảnh thành mảng NumPy
    fg_array = np.array(removed_bg)
    bg_array = np.array(bg_image)

    # Lấy kênh Alpha làm mặt nạ
    alpha = fg_array[:, :, 3] / 255.0  # Chuẩn hóa về [0, 1]

    # Kết hợp nền mới và foreground
    for c in range(3):  # Lặp qua R, G, B
        bg_array[:, :, c] = (1 - alpha) * bg_array[:, :, c] + alpha * fg_array[:, :, c]
    
    # Chuyển lại thành ảnh và lưu
    final_image = Image.fromarray(bg_array.astype("uint8"))
    final_image.save(output_path)
    print(f"Ảnh đã lưu tại {output_path}")

# Gọi hàm với ảnh đầu vào và nền mới
replace_background("000003.jpg", "pngtree-indoor-plant-on-wooden-floor-with-white-wall-background-image_15584292.jpg", "000003_1.png")